In [ ]:
#!pip install tensorflow-gpu==2.0.0b1
%load_ext tensorboard
%matplotlib inline

# Diagnosis with an auto-encoder

based on Arvin Singh Kushwaha https://towardsdatascience.com/how-to-make-an-autoencoder-2f2d99cd5103

In [ ]:
import tensorflow as tf
print(tf.__version__)

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Input, Flatten,\
                                    Reshape, LeakyReLU,\
                                    Activation, Dropout
from tensorflow.keras.models import Model, Sequential
from matplotlib import pyplot as plt
from IPython import display # If using IPython, Colab or Jupyter
import numpy as np

import datetime

from tensorflow.keras.callbacks import TensorBoard

In [ ]:
(x_train_all, y_train_all), (x_test_all, y_test_all) = mnist.load_data()

x_train_all = x_train_all/255.0
x_test_all = x_test_all/255.0

## Don't use everything: just a fraction of all available data
idx=np.random.choice(x_train_all.shape[0],int(0.1*x_train_all.shape[0]))

x_train = x_train_all[idx,:,:]
y_train = y_train_all[idx]

x_test = x_test_all
y_test = y_test_all

print("Training data with {0} samples".format(x_train.shape[0]))
print("Validation data with {0} samples".format(x_test.shape[0]))

# Plot image data from x_train
plt.imshow(x_train[0], cmap = "gray")
plt.show()

In [ ]:
LATENT_SIZE = 16
encoder = Sequential([
    Flatten(input_shape = (28, 28)), # 784D
    Dense(128),
    LeakyReLU(),
    #Dropout(0.5),
    Dense(64),
    LeakyReLU(),
    #Dropout(0.5),
    Dense(LATENT_SIZE, activation="sigmoid"),
])

In [ ]:
decoder = Sequential([
    Dense(64, input_shape = (LATENT_SIZE,)),
    LeakyReLU(),
    #Dropout(0.5),
    Dense(128),
    LeakyReLU(),
    Dense(784),
    Activation("sigmoid"),
    Reshape((28, 28))
])

In [ ]:
img = Input(shape = (28, 28))
latent_vector = encoder(img)
output = decoder(latent_vector)
model = Model(inputs = img, outputs = output)
model.compile("nadam", loss = "MeanAbsoluteError", metrics=['Accuracy','Precision','Recall'])

In [ ]:
log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

## TensorBoard Callback
tcb = TensorBoard(log_dir=log_dir, histogram_freq=1)

EPOCHS = 30

history_callback = model.fit(x_train, x_train, 
                             batch_size = 64, epochs=EPOCHS,
                             validation_data=(x_test, x_test),
                             callbacks=[tcb])

In [ ]:
%tensorboard --logdir logs/

In [ ]:
score = model.evaluate(x_test, x_test)
print('Test score:', score)

#acc = history_callback.history['accuracy']
loss = history_callback.history['loss']

#val_acc = history_callback.history['val_accuracy']
val_loss = history_callback.history['val_loss']

fig, (ax2) = plt.subplots(1,figsize=(8,6))
#ax1.plot(acc,label="Training Accuracy")
#ax1.plot(val_acc,label="Validation Accuracy")
ax2.plot(loss,label="Training Loss")
ax2.plot(val_loss,label="Validation Loss")

ax2.set_xlabel('epochs')
#ax1.legend()
ax2.legend()
plt.show()

In [ ]:
#Only do plotting if you have IPython, Jupyter, or using Colab
for epoch in range(EPOCHS):
    fig, axs = plt.subplots(4, 4, figsize=(4,4))
    rand = x_test[np.random.randint(0, 10000, 16)].reshape((4, 4, 1, 28, 28))
    
    #display.clear_output() # If you imported display from IPython
    
    for i in range(4):
        for j in range(4):
            axs[i, j].imshow(model.predict(rand[i, j])[0], cmap = "gray")
            axs[i, j].axis("off")
    
    plt.subplots_adjust(wspace = 0, hspace = 0)
    plt.show()
    print("-----------", "EPOCH", epoch, "-----------")
    model.fit(x_train, x_train, batch_size = 64)